# Purpose

The purpose of this notebook is to explore a simple regression tutorial with the [autokeras](https://autokeras.com/tutorial/structured_data_regression/) library.

# Data

The CA housing data will be used for this example, which is a simple regressiont task. It will be loaded from the sklearn data loader. I'll split off 20% into a test set which `autokeras` won't see during model development.

In [10]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import pandas as pd

housing = fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2
)

pd.DataFrame(X_train, columns=housing.feature_names).describe()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,3.866642,28.600291,5.429939,1.096732,1424.819828,2.999528,35.630778,-119.571148
std,1.888185,12.596480,2.564286,0.496689,1146.312518,4.461240,2.139404,2.006724
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.567300,18.000000,4.438304,1.006231,788.000000,2.436452,33.930000,-121.800000
50%,3.541700,29.000000,5.233582,1.048780,1162.500000,2.820659,34.250000,-118.500000
75%,4.739025,37.000000,6.051005,1.098941,1724.000000,3.286539,37.710000,-118.000000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,502.461538,41.950000,-114.310000


# StructuredDataRegressor

Now I'll use the `StructuredDataRegressor` class to automatically build the model without any architectural choices from me.

## Define and Fit

Below I'll simply point `autokeras` to a directory to save log files, give it a maximum number of search trials and let it commense a search.  You don't need to specify number of epochs as it will adapt to the task.

In [11]:
import autokeras as ak

automl = ak.StructuredDataRegressor(
    overwrite=True, max_trials=20, directory="./ak-logs/intro-ca/"
)

automl.fit(X_train, y_train)


Trial 20 Complete [00h 00m 52s]
val_loss: 0.4374975860118866

Best val_loss So Far: 0.42246752977371216
Total elapsed time: 00h 19m 39s
INFO:tensorflow:Oracle triggered exit



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/69
516/516 [==============================] - 1s 1ms/step - loss: 0.8458 - mean_squared_error: 0.8458
Epoch 2/69
516/516 [==============================] - 1s 1ms/step - loss: 0.5711 - mean_squared_error: 0.5711
Epoch 3/69
516/516 [==============================] - 1s 1ms/step - loss: 0.5355 - mean_squared_error: 0.5355
Epoch 4/69
516/516 [==============================] - 1s 1ms/step - loss: 0.5198 - mean_squared_error: 0.5198
Epoch 5/69
516/516 [==============================] - 1s 1ms/step - 

## Best Model

Now I'll export the best model and save it to hard disk before exploring and evaluating.

In [12]:
mod = automl.export_model()
mod.save("ak-mods/intro-ca")


INFO:tensorflow:Assets written to: ak-mods/intro-ca/assets


In [15]:
mod.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 8)                 0         
_________________________________________________________________
normalization (Normalization (None, 8)                 17        
_________________________________________________________________
dense (Dense)                (None, 256)               2304      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)               

## Test Model

In [16]:
from sklearn.metrics import mean_squared_error

train_predictions = mod.predict(X_train)
print("Train MSE:", mean_squared_error(y_train, train_predictions))
test_predictions = mod.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, test_predictions))


Train MSE: 0.3346768740720552
Test MSE: 0.396398391449601
